In [ ]:
!pip install -r requirements.txt

### <font color='red'>Setup</font> 
---
<font color='red'>⚠️ ⚠️ ⚠️</font> 
Before running this notebook, ensure you've run the [Set-Up Bedrock notebook](./set-up_bedrock.ipynb) notebook. <font color='red'>⚠️ ⚠️ ⚠️</font>

---

## Configure Bedrock

Create the necessary clients to invoke Bedrock models. If you need to pass in a certain role then set those values by uncommenting the section below.

First we instantiate using Anthropic Claude V2 for text generation, and Titan Embeddings G1 - Text for text embeddings.

Note: Many different models are available with Bedrock. Replace the `model_id` to change the model.

`llm = Bedrock(model_id="anthropic.claude-v2")`

Information on available model IDs [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import os
import sys
import boto3
import botocore

from langchain.llms.bedrock import Bedrock
from IPython.display import Image

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
# os.environ["AWS_PROFILE"] = ""
# os.environ["BEDROCK_ASSUME_ROLE"] = ""  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False)

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None))

model_parameter = {
    "temperature": 0.0, 
    "top_p": .5, 
    "top_k": 250, 
    "max_tokens_to_sample": 2000, 
    "stop_sequences": ["\n\n Human: bye"]
}
llm = Bedrock(
    model_id="anthropic.claude-v2", 
    model_kwargs=model_parameter, 
    client=bedrock_runtime
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)
Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Implementation
This notebook is using the LangChain framework where it has integrations with different services and the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V2 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: [S3FileLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.s3_file.S3FileLoader.html) and PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: In-Memory store FAISS

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

In [3]:
#TEST
customer_input = "I am  looking for a fan for our warehouses that will be energy efficient and assist with air circulation for the large open indoor warehouse. I need to be able to purchase in bulk for several locations in Indiana."


# Customer id to infuse order history and delivery address
customer_id = "2"

In [4]:
# Identify product attributes from customer prompt to generate better results
ner_prompt = """Human: Find industry, size, Sustainability Focus, Inventory Manager, and the location in the customer input.
Instructions:
The industry can be one of the following: Manufacturing, Warehousing, Government and Public Safety, Education, Food and Beverage Distribution, Hospitality, Property Management, Retail, or Other
The size can be one of the following: Small Businesses (Smaller companies might prioritize cost-effective solutions and fast shipping options), or Large Enterprises (Larger organizations may require more comprehensive solutions, including strategic services like inventory management and safety consulting), Womens, Other
The Sustainability Focused true or false meaning Environmentally Conscious Buyers: Customers interested in sustainability solutions, looking for products that focus on energy management, water conservation, waste reduction, and air quality improvement, or NOT Environmentally Conscious Buyers,
The Inventory Manager true or false meaning a purchaser in large amounts to supply an organizational group, versus an individual user purchasing for personal use, 
The output must be in JSON format inside the tags <attributes></attributes>

If the information of an entity is not available in the input then don't include that entity in the JSON output

Begin!

Customer input: {customer_input}
Assistant:"""
entity_extraction_result = llm(ner_prompt.format(customer_input=customer_input)).strip()
print(entity_extraction_result)

<attributes>
{
  "industry": "Warehousing",
  "size": "Large Enterprises", 
  "SustainabilityFocus": true,
  "InventoryManager": true,
  "location": "Indiana"
}
</attributes>


#### Extract values into JSON

In [5]:
import re
import json
result = re.search('<attributes>(.*)</attributes>', entity_extraction_result, re.DOTALL)
attributes = json.loads(result.group(1))
attributes

{'industry': 'Warehousing',
 'size': 'Large Enterprises',
 'SustainabilityFocus': True,
 'InventoryManager': True,
 'location': 'Indiana'}

## Use Retrieval Augmented Generation (RAG) 

Note: documents loaded with [S3FileLoader available under LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/) can be split into smaller chunks. The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 8k tokens, which roughly translates to ~32000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

Below: fetching our productdata and creating the embeddings for 
1. Product catalog description
2. Customer reviews

# TODO: ADD order history for logged in user
3. Order History 

In [13]:
parquet_file_path = "processed/grainger_products.parquet"
print("Attempting to load file from:", parquet_file_path)

# Now attempt to load the file
try:
    df = pd.read_parquet(parquet_file_path)
    print("File loaded successfully!")
except FileNotFoundError as e:
    print("Error loading file:", e)

print(df.head())

Attempting to load file from: processed/grainger_products.parquet
File loaded successfully!
                          Brand    Code  \
0  LION FIRE BOOTS BY THOROGOOD   3XRG7   
1           GLOWEAR BY ERGODYNE   1CXK5   
2                      CARHARTT  491V68   
3                      TRIPLETT  794UC5   
4                        DEWALT  492U19   

                                                Name  \
0  Insulated Firefighter Boots: Insulated, Steel,...   
1  GLOWEAR BY ERGODYNE Baseball Cap: Orange, Univ...   
2  CARHARTT Bib Overalls: Men's, XL ( 42 in x 32 ...   
3  TRIPLETT Combustible Gas Detector: Audible/Vib...   
4  DEWALT Heated Jacket: Men's, S, Black, Up to 9...   

                                       PictureUrl600    Price  \
0  https://static.grainger.com/rp/s/is/image/Grai...  $197.55   
1  https://static.grainger.com/rp/s/is/image/Grai...   $13.93   
2  https://static.grainger.com/rp/s/is/image/Grai...   $95.79   
3  https://static.grainger.com/rp/s/is/image/Grai...

In [ ]:
## Approach 1: Using Textual Columns Directly
# To focus on text-based similarity (e.g., based on 'Name' and 'Description'), columns are concatenated
# FAISS.from_texts is used. This allows more granular control.
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd

# Initialize the Titan Embeddings Model
print("Initializing Titan Embeddings Model...")
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_runtime)
print("Titan Embeddings Model initialized.")

# Load the Grainger products data from parquet file at path relative to the notebook
parquet_file_path = "processed/grainger_products.parquet"
print(f"Loading data from parquet file: {parquet_file_path}...")
products_df = pd.read_parquet(parquet_file_path)
print("Data loaded successfully.")

# Concatenate 'Name', 'Description', and 'Code' columns
print("Concatenating 'Name', 'Description', and 'Code' columns...")
products_df['text_content'] = products_df['Name'].fillna('') + ' ' + products_df['Description'].fillna('') + ' ' + products_df['Code'].fillna('')

# Create FAISS vector store from concatenated text content
print("Creating FAISS vector store based on concatenated text content...")
vectorstore_faiss = FAISS.from_texts(products_df['text_content'], bedrock_embeddings)
print("FAISS vector store created.")

# Display count of vectors added during creation
print(f"Number of vectors in the FAISS vector store: {vectorstore_faiss.num_vectors}")

# Print the dimensionality of the vectors in the FAISS vector store
print(f"Vector dimensionality in FAISS vector store: {vectorstore_faiss.vector_dim}")

# TEST: Process a query based on a product code
customer_code = "1CXK5"  # Example product code to search
print(f"Processing customer query for product with code: {customer_code}...")
query_result = products_df[products_df['Code'] == customer_code]

if not query_result.empty:
    print("Product details found:")
    print(query_result)
else:
    print(f"No product found with code: {customer_code}")

# TEST: Process a query based on customer input
customer_input = "Men's insulated boots"
print(f"Processing customer input: {customer_input}...")
query_embedding = vectorstore_faiss.embedding_function(customer_input)
print("Customer input processed.")

# Convert query embedding to numpy array
np_array_query_embedding = np.array(query_embedding)
print("Query embedding converted to numpy array.")

# Print the resulting query embedding
print("Resulting query embedding:")
print(np_array_query_embedding)


Initializing Titan Embeddings Model...
Titan Embeddings Model initialized.
Loading data from parquet file: processed/grainger_products.parquet...
Data loaded successfully.
Concatenating 'Name', 'Description', and 'Code' columns...
Creating FAISS vector store based on concatenated text content...


In [ ]:
#VERSION 2: AS A DOCUMENT
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd

# Initialize the Titan Embeddings Model
print("Initializing Titan Embeddings Model...")
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_runtime)
print("Titan Embeddings Model initialized.")

# Example: Creating structured documents
documents = [
    {
        'content': f"{row['Code']} {row['Name']} {row['Brand']} {row['Description'] if pd.notna(row['Description']) else ''}",  # Concatenated text content
        'metadata': {
            'Brand': row['Brand'],
            'Code': row['Code'],
            'Name': row['Name'],
            'Description': row['Description'],
            'Price': row['Price']
            # Add other metadata fields as needed
        }
    }
    for _, row in df.iterrows()
]

# Print the structured documents
print("Structured documents created:")
for doc in documents:
    print(doc)

# Create FAISS vector store from structured documents
print("Creating FAISS vector store from structured documents...")
vectorstore_faiss_doc = FAISS.from_documents(documents, bedrock_embeddings)
print("FAISS vector store created.")

# Example: Print some details about the created vector store
print(f"Number of vectors in the FAISS vector store: {len(vectorstore_faiss_doc)}")

# Assuming `customer_input` is defined elsewhere in your code
# Replace this with actual customer input or query as needed
customer_input = "Men's insulated boots"
print(f"Processing customer input: {customer_input}...")
query_embedding_doc = vectorstore_faiss_doc.embedding_function(customer_input)
print("Customer input processed.")

# Convert query embedding to numpy array
np_array_query_embedding_doc = np.array(query_embedding_doc)
print("Query embedding converted to numpy array.")

# Print the resulting query embedding
print("Resulting query embedding:")
print(np_array_query_embedding_doc)


## Generate *`n`* style recommendations

Make a query to embed the LLM using customer input. Using LangChain for orchestration of RAG. It also provides a framework for orchestrating RAG flows with what purpose built "chains". In this section, we will see how to be a [retrieval chain](https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa) which is more comprehensive and robust than the original retrieval system we built above.

The workflow we used above follows the following process:
1. User input is received.
2. User input is queried against the vector database to retrieve the relevant products
3. Product description and chat memory are inserted into a new prompt to respond to the user input.
4. This output is fed into the stable diffusion model to return the relevant images

However, more complex methods of interacting with the user input can generate more accurate results in RAG architectures. One of the popular mechanisms which can increase accuracy of these retrieval systems is utilizing more than one call to an LLM in order to reformat the user input for more effective search to your vector database. A better workflow is described below compared to the one we already built...

1. User input is received.
2. An LLM is used to reword the user input to be a better search query for the vector database based on the chat history and product description. 
3. This could include things like condensing, rewording, addition of chat context, or stylistic changes.
4. Reformatted user input is queried against the vector database to retrieve relevant products.The reformatted user input and relevant documents are inserted into a new prompt in order to generate the new style. 
5. This is then fed into the stable diffusion model to generate the images. 

In your application the images can come from a pre canned images 

We will now build out this second workflow using LangChain below. First we need to make a prompt which will reformat the user input to be more compatible for searching of the vector database. The way we do this is by providing the chat history as well as the some basic instructions to Claude and asking it to condense the input into a single output. 

In [13]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to generate 5 style recommendations for the customer input at the end.
<context>
{context}
</context>
<example>A navy suit with a light blue dress shirt, conservative tie, black oxford shoes, and a leather belt.</example>
<example>A lehenga choli set with a crop top, flowing skirt, and dupatta scarf in lively colors and metallic accents.</example>

Customer Input: {question}
Each style recommendation must be inside the tags <style></style>.
Do not output product physical IDs.
Skip the preamble.
Assistant: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Use RetrievalQA customizations for imprving Q&A experience
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 6}
    ),
    return_source_documents=False,
    chain_type_kwargs={"prompt": PROMPT},
)
styles_response = qa({"query": customer_input})['result']

# Alternitively we can query using wrapper also
# styles_response = wrapper_store_faiss.query(question= customer_input, llm=llm)

print_ww(styles_response)

 Unfortunately the customer input does not seem to match the provided context, which is about
clothing and fashion recommendations. The input is asking about purchasing industrial fans for
warehouses. I do not have enough relevant context to generate clothing style recommendations based
on this input. Please provide clothing/fashion related customer input to match the provided context.


### Prepare the received response

Since we have instructed LLM to return our data is returned as XML wrapping a JSON, we run the necessary extraction steps to fetch the relevant details to generate images for each look. 

In [14]:
# Prepare input to fetch images for each look
styles = re.findall('<style>(.*?)</style>', styles_response)
styles

[]

## Generate Images for the relevant style

Generate an image for each look using the `Stable Diffusion` model

![Generate Look](./images/generate_look.png)

In [9]:
from PIL import Image
from IPython import display
from base64 import b64decode
import base64
import io
import json
import os
import sys
import ipywidgets as widgets

# Fetching images for each of style
gender_map = {
    'Womens': 'of a female ',
    'Mens': 'of a male '
}

os.makedirs("data", exist_ok=True)
image_strip = ""
for i, style in enumerate(styles):
    request = json.dumps({
        "text_prompts": [
            {"text": f"Full body view {gender_map.get(attributes.get('gender'))}without a face in " + style + "dslr, ultra quality, dof, film grain, Fujifilm XT3, crystal clear, 8K UHD", "weight": 1.0},
            {"text": "poorly rendered", "weight": -1.0}
        ],
        "cfg_scale": 9,
        "seed": 4000,
        "steps": 50,
        "style_preset": "photographic",
    })
    modelId = "stability.stable-diffusion-xl"
    
    response = bedrock_runtime.invoke_model(body=request, modelId=modelId)
    response_body = json.loads(response.get("body").read())
    
    base_64_img_str = response_body["artifacts"][0].get("base64")
    # display.display(display.Image(b64decode(base_64_img_str), width=200))
    image_strip += "<td><img src='data:image/png;base64, "+ base_64_img_str + "'></td>"

display.display(display.HTML("<table><tr>" + image_strip +"</tr></table>"))

## Enhance user experience with Chatbot

#### Generating detailed overview based on customer reviews of products in catalog 
We have discussed the key building blocks needed for the chatbot application and now we will start to create them. LangChain's [ConversationBufferMemory](https://python.langchain.com/docs/use_cases/question_answering/chat_vector_db) class provides an easy way to capture conversational memory for LLM chat applications. We will have Claude being able to retrieve context through conversational memory using the prompt template. Note that this time our prompt template includes a {chat_history} variable where our chat history will be included to the prompt.

The prompt template has both conversation memory as well as chat history as inputs along with the human input. Notice how the prompt also instructs Claude to not answer questions which it does not have the context for. This helps reduce hallucinations which is extremely important when creating end user facing applications which need to be factual.


![Architecture](./images/chatbot_products.png)

In [11]:
chat_prompt1 = "Show me specific reviews that talk about the quality of the fabric for the jacket."
chat_prompt2 = "What do people like about the business formal jacket?"

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

chat_history = [" "]
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=vectorstore_faiss.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# Generate detailed reviews based on customer reviews of specific clothing in product catalog

try:
    chat_res1 = conversation.run({'question': chat_prompt1, 'chat_history': chat_history })
    print_ww(chat_res1)
    chat_history.append([chat_prompt1, chat_res1])
except ValueError as error:
    if  "AccessDeniedException" in str(error):
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [12]:
try:
    chat_res2 = conversation.run({'question': chat_prompt2 + " Answer even if embeddings does not return anything.", 'chat_history': chat_history })
    print_ww(chat_res2)
    chat_history.append([chat_prompt2, chat_res2])
except ValueError as error:
    if  "AccessDeniedException" in str(error):
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

#### Customer order history semantic searches

Generating size and color recommendations based on customer's order history. This will help to provide curated content to the customer

In [58]:
chat_prompt3 = "What size and color should I wear?"
chat_res3 = wrapper_store_faiss.query(question= chat_prompt3 + " based on order history for customer with id " + customer_id, llm=llm)
print_ww(chat_res3)

## Showing final products based on customer style selection 

Continuing on our architectural pattern we will change the prompt template and leverage the LLM to generate the `recommended` products based on the user selection and weather and other details. The key extraction entities will be 

1. Leverage the customer initial prompt to generate the relevant ids
2. Extract the relevant products from the vector store
3. Physical ID for the products needed



![Architecture](./images/other_products.png)

In [ ]:
from PIL import Image
import requests

prompt_template2 = """Human: Extract list of products and their respective physical IDs from catalog that matches the style given below. 
The catalog of products is provided under <catalog></catalog> tags below.
<catalog>
{context}
</catalog>
Style: {question}

The output should be a JSON of the form <products>[{{"product": <description of the product from the catalog>, "physical_id":<physical id of the product from the catalog>}}, ...]</products>
Skip the preamble.
Assistant: """

PROMPT2 = PromptTemplate(
    template=prompt_template2, input_variables=["context", "question"]
)
qa2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 10}
    ),
    chain_type_kwargs={"prompt": PROMPT2},
    return_source_documents=True,
)

selected_style = styles[3]
print(selected_style)
cart_items = qa2({"query": selected_style })['result']
print_ww(cart_items)

In [15]:
products = json.loads(re.findall('<products>(.*?)</products>', cart_items, re.DOTALL)[0])
products

In [16]:
from PIL import Image
from IPython import display
import requests
import urllib.parse

cart_item_strip = ""
for product in products:
    url = "https://sagemaker-example-files-prod-us-east-1.s3.us-east-1.amazonaws.com/datasets/image/howser-bedrock/data/aistylist/images/products/" + urllib.parse.quote(product['physical_id'].strip(), safe='', encoding=None, errors=None) + ".jpg"
    # im = Image.open(requests.get(url, stream=True).raw)
    cart_item_strip += "<td><img src='"+ url + "'></td>"
display.display(display.HTML("<table><tr>" + cart_item_strip +"</tr></table>"))

## Integrating DIY Agents to associate external APIs and databases
### Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses. Here we will leverage the user prompt and perform the following actions
1. Extract the city 
2. Get weather information
3. Search our product catalog using semantic search to find relevant products
4. Display the products for user to add to cart

![Architecture](./images/weather.png)

In [17]:
import os
import python_weather

async def getweather(city):
  # declare the client. the measuring unit used defaults to the metric system (celcius, km/h, etc.)
  async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
    # fetch a weather forecast from a city
    weather = await client.get(city)
    
    # returns the current day's forecast temperature (int)
    return weather.current

## Accessory recommendations 

We will provide  accessory recommendations based on location provided in customer input

In [18]:
await getweather(entity_extraction_result[2])

In [19]:
import asyncio
accessory_response = None
if attributes["location"]:
    current_weather = await getweather(entity_extraction_result[2])
    accessory_input = "Suggest list of accessories based on the weather and the selected style. It is " + current_weather.description + " with temperature at " + str(current_weather.temperature) +" degrees fahrenheit.\n Selected Style: " + styles[0]
    accessory_response = qa({"query": accessory_input})['result']
    print_ww(accessory_response)

In [20]:
if accessory_response:
    accessories = re.findall('<style>(.*?)</style>', accessory_response)
    accessories_items = qa2({"query": ', '.join(accessories)})['result']
    accessories_items = json.loads(re.findall('<products>(.*?)</products>', accessories_items, re.DOTALL)[0])
    accessory_strip = ""
    for accessory in accessories_items:
        url = "https://sagemaker-example-files-prod-us-east-1.s3.us-east-1.amazonaws.com/datasets/image/howser-bedrock/data/aistylist/images/products/" + urllib.parse.quote(accessory['physical_id'].strip(), safe='', encoding=None, errors=None) + ".jpg"
        accessory_strip += "<td><img src='"+ url + "'></td>"
    display.display(display.HTML("<table><tr>" + accessory_strip +"</tr></table>"))

## Simulate the order check out

Add a customer data table to complete the order transaction. This information provides the shipping address for the outfit order.

In [21]:
customer_table=[{"id": 1, "first_name": "John", "last_name": "Doe", "age": 35, "address": "123 Bedrock st, California 90210"},
  {"id": 2, "first_name": "Jane", "last_name": "Smith", "age": 27, "address": "234 Sagemaker drive, Texas 12345"},
  {"id": 3, "first_name": "Bob", "last_name": "Jones", "age": 42, "address": "111 DeepRacer ct, Virginia 55555"},
  {"id": 4, "first_name": "Sara", "last_name": "Miller", "age": 29, "address": "222 Robomaker ave, New Yotk 13579"},
  {"id": 5, "first_name": "Mark", "last_name": "Davis", "age": 31, "address": "444 Transcribe blvd, Florida 02468"},
  {"id": 6, "first_name": "Laura", "last_name": "Wilson", "age": 24, "address": "555 CodeGuru st, California 98765" },
  {"id": 7, "first_name": "Steve", "last_name": "Moore", "age": 36, "address": "456 DeepLens st, Texas 11223"},
  {"id": 8, "first_name": "Michelle", "last_name": "Chen", "age": 22, "address": "642 DeepCompose st, Colorado 33215"},
  {"id": 9, "first_name": "David", "last_name": "Lee", "age": 29, "address": "777 S3 st, California 99567"},
  {"id": 10, "first_name": "Jessica", "last_name": "Brown", "age": 18, "address": "909 Ec st, Utah 43210"}]

def address_lookup(id):
    for customer in customer_table:
        if customer["id"] == int(id):
            return customer
        
    return None

print(address_lookup(customer_id)["address"])